## Постановка задачи
Загрузим данные, приведем их к числовым, заполним пропуски, нормализуем данные и оптимизируем память.

Разделим выборку на обучающую/проверочную в соотношении 80/20.

Построим XGBoost модель.

Проведем предсказание и проверим качество через каппа-метрику.

Данные:
* https://video.ittensive.com/machine-learning/prudential/train.csv.gz

Соревнование: https://www.kaggle.com/c/prudential-life-insurance-assessment/

© ITtensive, 2020

In [1]:
GRAIN = 11
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, confusion_matrix, make_scorer
from sklearn.model_selection import GridSearchCV, cross_val_score
from xgboost import XGBClassifier
from sklearn import preprocessing
from etl_utils import reduce_mem_usage


data = pd.read_csv("https://video.ittensive.com/machine-learning/prudential/train.csv.gz")

data['Product_Info_2_1'] = data['Product_Info_2'].str.slice(0, 1)
data['Product_Info_2_2'] = pd.to_numeric(data['Product_Info_2'].str.slice(1, 2))
data = data.drop('Product_Info_2', axis='columns')

onehot_df = pd.get_dummies(data['Product_Info_2_1'])
onehot_df.columns = ['Product_Info_2_1' + column for column in onehot_df.columns]
data = pd.merge(left=data, right=onehot_df, left_index=True, right_index=True).drop('Product_Info_2_1', axis=1).fillna(-1)
del onehot_df

data['Response'] = data['Response'] - 1

### Набор столбцов для расчета

In [2]:
columns_groups = ['Insurance_History', 'InsurеdInfo', 'Medical_Keyword', 'Family_Hist', 'Medical_History', 'Product_Info']
columns = ['Wt', 'Ht', 'Ins_Age', 'BMI']
for cg in columns_groups:
    columns.extend(data.columns[data.columns.str.startswith(cg)])
print(columns)

['Wt', 'Ht', 'Ins_Age', 'BMI', 'Insurance_History_1', 'Insurance_History_2', 'Insurance_History_3', 'Insurance_History_4', 'Insurance_History_5', 'Insurance_History_7', 'Insurance_History_8', 'Insurance_History_9', 'Medical_Keyword_1', 'Medical_Keyword_2', 'Medical_Keyword_3', 'Medical_Keyword_4', 'Medical_Keyword_5', 'Medical_Keyword_6', 'Medical_Keyword_7', 'Medical_Keyword_8', 'Medical_Keyword_9', 'Medical_Keyword_10', 'Medical_Keyword_11', 'Medical_Keyword_12', 'Medical_Keyword_13', 'Medical_Keyword_14', 'Medical_Keyword_15', 'Medical_Keyword_16', 'Medical_Keyword_17', 'Medical_Keyword_18', 'Medical_Keyword_19', 'Medical_Keyword_20', 'Medical_Keyword_21', 'Medical_Keyword_22', 'Medical_Keyword_23', 'Medical_Keyword_24', 'Medical_Keyword_25', 'Medical_Keyword_26', 'Medical_Keyword_27', 'Medical_Keyword_28', 'Medical_Keyword_29', 'Medical_Keyword_30', 'Medical_Keyword_31', 'Medical_Keyword_32', 'Medical_Keyword_33', 'Medical_Keyword_34', 'Medical_Keyword_35', 'Medical_Keyword_36', 'M

### Нормализация данных

In [3]:
scaler = preprocessing.StandardScaler()
data_transformed = pd.DataFrame(scaler.fit_transform(data[columns]))
columns_transformed = data_transformed.columns
data_transformed['Response'] = data['Response']
data_transformed = reduce_mem_usage(data_transformed)
data_transformed.info()

Потребление памяти меньше на 40.49 Мб (-75.1%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59381 entries, 0 to 59380
Columns: 119 entries, 0 to Response
dtypes: float16(118), int8(1)
memory usage: 13.4 MB


### Разделение данных
Преобразуем выборки в отдельные наборы данных

In [4]:
data_train, data_test = train_test_split(data_transformed, test_size=.2, random_state=GRAIN)
data_train.head()

,0,1,2,3,4,5,6,7,8,9,...,109,110,111,112,113,114,115,116,117,Response
40387,-0.090637,1.249023,-0.618652,-0.786133,-1.634766,-0.169434,0.862305,-1.013672,0.880859,-0.928711,...,11.945312,0.441650,-0.149292,-0.666992,-0.623535,-0.14209,-0.128906,0.750977,-0.215942,7
17090,-0.513672,-0.220581,0.062622,-0.481445,0.611816,-0.169434,0.862305,-1.013672,0.867676,-0.928711,...,-0.083679,0.441650,-0.149292,-0.200073,-0.623535,-0.14209,-0.128906,0.750977,-0.215942,6
21687,0.426270,1.003906,1.046875,-0.081970,-1.634766,-0.169434,0.862305,-1.013672,0.869629,-0.928711,...,-0.083679,0.441650,-0.149292,-1.133789,-0.623535,-0.14209,-0.128906,0.750977,-0.215942,7
40073,0.567383,1.003906,1.878906,0.075378,0.611816,-0.169434,0.862305,0.043671,0.861328,-0.928711,...,-0.083679,0.441650,-0.149292,-1.133789,-0.623535,-0.14209,-0.128906,0.750977,-0.215942,5
3408,-0.395996,-0.220581,0.516602,-0.329834,0.611816,-0.169434,0.862305,0.043671,0.867676,-0.928711,...,-0.083679,-2.263672,-0.149292,-0.200073,-0.623535,-0.14209,-0.128906,0.750977,-0.215942,6


### XGBoost = градиентный бустинг деревьев решений
![](https://static.learme.ru/storage/uploads/editor/VOKWrS5HbJvj1HXwgJWVCbPee9leV6B5YXvAsbcy.png)

Строится решающее дерево, проверяется, после этого строится новое дерево, которое исправляет ошибки первого (используется последовательное применение моделей). И так - несколько итераций или до достижения нужной точности предсказания.

Основное отличие XGBoost от обычного градиентного бустинга - в использовании второй производной ошибки, из-за этого сходимость ансамбля лучше. Также XGBoost позволяет распараллелить вычисления.

Будем использовать параметры случайного леса для модели.

In [5]:
%%time
model = XGBClassifier(
    max_depth=15, n_estimators=76 #, max_features=27, min_samples_leaf=19
).fit(data_train[columns_transformed], data_train['Response'])

CPU times: total: 11min 10s
Wall time: 48.1 s


### Предсказание данных и оценка модели

In [6]:
data_test['target'] = model.predict(data_test[columns_transformed])

Кластеризация дает 0.192, kNN(100) - 0.3, лог. регрессия - 0.512/0.496, SVM - 0.95, реш. дерево - 0.3, случайный лес - 0.487

In [7]:
print("XGBoost:", round(cohen_kappa_score(data_test["target"], data_test["Response"], weights="quadratic"), 3))

XGBoost: 0.534


### Матрица неточностей

In [8]:
print("XGBoost\n", confusion_matrix(data_test["target"], data_test["Response"]))

XGBoost
 [[ 267  164   16   13   66  112   48   33]
 [ 193  298   16    7   92  110   38   27]
 [  20   27   75   21    2    0    0    0]
 [  42   30   60  215    0    5    0    1]
 [ 108  141    8    0  582  107   13   14]
 [ 236  252   19   21  172 1178  246  158]
 [ 139  137    3    6   62  282  722  209]
 [ 233  258    5   33   88  442  495 3510]]
